this nb is responsible for scraping the data and handling all the messy html stuff. gives you a csv file containing the pq paragraphs and a bit of metadata.

sometimes code is too fast for browser driver. in such cases, try increasing sleep time.

In [1]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
import math
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
from enum import Enum
import re
import pandas as pd
 

In [2]:
class ReportSection(Enum):
    WRITTEN = 'Written Answers to Questions'
    WRITTEN_NA = 'Written Answers to Questions for Oral Answer Not Answered by End of Question Time'
    ORAL = 'Oral Answers to Questions'

In [3]:
DRIVER_PATH = 'chromedriver_win32/chromedriver.exe'
SEARCH_TERM = 'asked the' # can be blank
# PARLIAMENT_INDEX = -1 # legislative assembly = 1, 1st parliament = 2, etc etc. -1 to ignore this field.

In [4]:
def element_completely_viewable(driver, elem):
    elem_left_bound = elem.location.get('x')
    elem_top_bound = elem.location.get('y')
    elem_width = elem.size.get('width')
    elem_height = elem.size.get('height')
    elem_right_bound = elem_left_bound + elem_width
    elem_lower_bound = elem_top_bound + elem_height

    win_upper_bound = driver.execute_script('return window.pageYOffset')
    win_left_bound = driver.execute_script('return window.pageXOffset')
    win_width = driver.execute_script('return document.documentElement.clientWidth')
    win_height = driver.execute_script('return document.documentElement.clientHeight')
    win_right_bound = win_left_bound + win_width
    win_lower_bound = win_upper_bound + win_height

    return all((win_left_bound <= elem_left_bound,
                win_right_bound >= elem_right_bound,
                win_upper_bound <= elem_top_bound,
                win_lower_bound >= elem_lower_bound)
              )

def find_element(driver, elem):
    y_height = 0
    while not element_completely_viewable(driver, elem):
        driver.execute_script(f"window.scrollTo(0, {y_height})") 
        y_height += 300
        time.sleep(0.25)

def find_and_click(driver, elem):
    find_element(driver, elem)
    if element_completely_viewable(driver, elem):
        elem.click()
    else:
        print('finding and clicking agn')
        find_and_click(driver, elem)



In [5]:
driver = webdriver.Chrome(DRIVER_PATH) 
 
page_url = 'https://sprs.parl.gov.sg/search/home' 
driver.get(page_url) 
driver.maximize_window() 
driver.implicitly_wait(20) 
time.sleep(2) 
 
# Get search box and fill it up 

search = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[1]/div/div[1]/input')
search.send_keys(SEARCH_TERM) 

# exact phrase search

search_opt = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[2]/div/div[1]/select')
find_and_click(driver, search_opt)
exact_search_opt = driver.find_element(by=By.XPATH, value='//*[@id="divmpscreen2"]/div[2]/div[2]/div/div[1]/select/option[3]')
find_and_click(driver, exact_search_opt)
 
# Uncomment following two lines to only search in titles # 
#checkbox = driver.find_element_by_xpath('//*[@id="divmpscreen2"]/div[2]/div[1]/div/label/input')
#checkbox.click() 

# from 1st parliament to 14th. ignore legis. assembly cuz the parl website has no list of members.
for i in range(2, 16):
    session = driver.find_element(
        by=By.XPATH, 
        value=f'/html/body/app-root/app-search/div/div[2]/div[2]/div[1]/div/div[3]/select/option[{i}]')
    session.click() 
 
# advanced search - only look for oral and written answers to questions

adv_search_opt = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[3]/div/div[1]/h5/a')
find_and_click(driver, adv_search_opt)
section_selection = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[3]/div/div[2]/div/div/div/div/div[3]/div/select')
find_element(driver, section_selection)
for selection_option_index in [13, 19, 20]: # corresponds to answers to questions    
    option = driver.find_element(
        by=By.XPATH, 
        value=f'/html/body/app-root/app-search/div/div[3]/div/div[2]/div/div/div/div/div[3]/div/select/option[{selection_option_index}]')
    option.click()


# Find submit element and press enter

submit = driver.find_element(by=By.XPATH, value='/html/body/app-root/app-search/div/div[2]/div[2]/div[3]/div/button[2]') 
submit.send_keys(Keys.RETURN) 

print('Search parameters submitted.') 

# Switch window and check for number of search results 

driver.switch_to.window(driver.window_handles[1])
driver.implicitly_wait(20) 
time.sleep(2) 

try:
    num_res_label = driver.find_element(by=By.CLASS_NAME, value='showingResults')
except: 
    print('Error when finding num_res_label')
    
num_res_text = num_res_label.get_attribute('innerHTML')
    
if 'No' in num_res_text:
    print('no results found') # but site is quite erratic, may take a few tries to get results (idk why)
else:
    num_results = int(num_res_text[num_res_text.rfind(' ')+1:])
    num_pages = math.ceil(num_results / 20)
    print(f'{num_results} results found spanning {num_pages} page(s)')

C:\Users\thisi\AppData\Local\Temp\ipykernel_23620\1286528577.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(DRIVER_PATH)


Search parameters submitted.
11904 results found spanning 596 page(s)


In [6]:
pq_paras = pd.read_csv('pq_paras.csv').values.tolist()
print(len(pq_paras))
docs_saved = 0

5299


In [ ]:
docs_to_skip = 4881 # should set to 0. use this if it crashes halfway.
docs_saved = 0
pages_seen = 0

while (True):
    driver.switch_to.window(driver.window_handles[1])
    
    if docs_to_skip < 20:

        elems = driver.find_elements(by=By.XPATH, value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/table/tbody')
        links = list(map(lambda elem: elem.find_elements(by=By.XPATH, value='.//tr[1]/td[2]/a')[0], elems))

        for link in links:
            if docs_to_skip > 0:
                docs_to_skip -= 1
                docs_saved += 1
                continue
                
            try:
                find_and_click(driver, link)

                # Switch to page with content and get URL name
                time.sleep(1)
                while len(driver.window_handles) < 3:
                    find_and_click(driver, link)
                    print(f'trying this link agn')
                    time.sleep(2)

                driver.switch_to.window(driver.window_handles[2])         
                item_key = driver.current_url.split('/')[-1]         
                item_key = item_key.replace('?', '_') # Replace ? because it would be an invalid filename 

                file = item_key
                page_source = driver.page_source

                if file[0:4] == 'sprs':
                    is_sprs = True
                else:
                    is_sprs = False

                soup = bs(page_source, 'html.parser')
                
                if not soup.find('table'):
                    # trust me, this happens. do exact phrase search for "MISSELLING OF STRUCTURED PRODUCTS BY FINANCIAL INSTITUTIONS (UPDATE ON INVESTIGATION BY MAS)"
                    print(f"can't find a single table, page is probably empty {driver.current_url}")
                    docs_saved += 1
                    driver.close()
                    driver.switch_to.window(driver.window_handles[1])
                    time.sleep(1)
                    continue

                if is_sprs:
                    # extract metadata from the table at the top of the page
                    topic_table = soup.find('table', {'class': 'topic'})
                    topic_text = topic_table.get_text()

                    parl_no_label_span = re.match(r'Parliament No:', topic_text).span()
                    topic_text = topic_text[parl_no_label_span[1]:]
                    parl_no = int(re.match(r'\d+', topic_text).group())

                    sit_date_label_span = re.match(r'.*Sitting Date:', topic_text).span()
                    topic_text = topic_text[sit_date_label_span[1]:]
                    sit_date = datetime.strptime(re.match(r'\d+-\d+-\d+', topic_text).group(), '%d-%m-%Y')

                    # extract report section via filename
                    if 'oral-answer' in file:
                        section = ReportSection.ORAL
                    elif 'written-answer-na' in file:
                        section = ReportSection.WRITTEN_NA
                    elif 'written-answer' in file:
                        section = ReportSection.WRITTEN
                    else:
                        raise 'unidentified section (sprs)'
                    section = section.value

                    # extract paragraphs from report in the form of rows of table
                    report_table = soup.find('div', {'class': 'reportTable'})
                    report_table_rows = report_table.findChildren()

                    # find out which rows correspond to pqs and which rows don't.
                    for row in report_table_rows:
                        paragraph = row.get_text()
                        paragraph = paragraph.replace('\xa0', ' ')
                        if not re.match(r'\d+.{8,75} asked the .+', paragraph):
                            continue
                        paragraph_no_num = paragraph[
                            re.search('\d+', paragraph).span()[1]:]
                        pq_paras.append((paragraph_no_num.strip(), sit_date, parl_no, section))
                else:
                    # extract metadata from meta elements directly
                    sit_date_elem = soup.find('meta', {'name': 'Sit_Date'})
                    parl_no_elem = soup.find('meta', {'name': 'Parl_No'})
                    section_elem = soup.find('meta', {'name': 'Sect_Name'})
                    
                    if sit_date_elem:
                        sit_date_str = sit_date_elem['content']
                        sit_date = datetime.strptime(sit_date_str, '%Y-%m-%d')
                    else:
                        # i didn't wanna have this block but apparently the meta element isn't always there
                        sit_date_str = driver.find_element(
                            by=By.XPATH, 
                            value='/html/body/app-root/app-topic/div[2]/div/div/div/div/div[2]/div/div/div/div/div/table/tbody/tr/td/table/tbody/tr[5]/td[2]/font'
                        ).get_attribute('innerHTML')
                        sit_date = datetime.strptime(sit_date_str, '%d-%m-%Y')
                        
                    if parl_no_elem:
                        parl_no_str = parl_no_elem['content']
                    else:
                        # i didn't wanna have this block but apparently the meta element isn't always there
                        parl_no_str = driver.find_element(
                            by=By.XPATH, 
                            value='/html/body/app-root/app-topic/div[2]/div/div/div/div/div[2]/div/div/div/div/div/table/tbody/tr/td/table/tbody/tr[1]/td[2]/font'
                        ).get_attribute('innerHTML')
                        
                    if section_elem:
                        section_str = section_elem['content']
                    else:
                        # i didn't wanna have this block but apparently the meta element isn't always there
                        section_str = driver.find_element(
                            by=By.XPATH, 
                            value='/html/body/app-root/app-topic/div[2]/div/div/div/div/div[2]/div/div/div/div/div/table/tbody/tr/td/table/tbody/tr[6]/td[2]/font'
                        ).get_attribute('innerHTML')

                    parl_no = int(parl_no_str)
                    if 'NOT REACHED' in section_str or 'Not' in section_str:
                        section = ReportSection.WRITTEN_NA
                    elif 'ORAL' in section_str or 'Oral' in section_str:
                        section = ReportSection.ORAL
                    elif 'WRITTEN' in section_str or 'Written' in section_str:
                        section = ReportSection.WRITTEN
                    else:
                        raise 'unidentified section (non sprs)'
                    section = section.value

                    # extract paragraphs from table in the form of raw text (since each paragraph doesn't have its own element)
                    report_table = soup.find('div', {'class': 'reportTable'})
                    if not report_table:
                        print('no report table')
                        raw_text = soup.get_text()
                    else:
                        raw_text = report_table.get_text()

                    # look for things like 1. 2. etc
                    first_number_dot_occurrence = re.search(r'\d+\.(\s+)(.|\s){8,75}(\s+)asked(\s+)the(\s+)', raw_text)

                    # cuz apparently sprs doesn't guarantee a dot being there.
                    if not first_number_dot_occurrence:
                        first_number_dot_occurrence = re.search(r'\d+(\s+)(.|\s){8,75}(\s+)asked(\s+)the(\s+)', raw_text)

                    # cut off everything before the actual body of the text
                    raw_text = raw_text[first_number_dot_occurrence.span()[0]:]

                    # formatting stuff
                    if '\xa0' in raw_text:
                        raw_text = raw_text.replace('\n\n', ' ')
                        raw_paras = raw_text.split('\xa0\xa0\xa0\xa0')
                    else:
                        raw_paras = raw_text.split('\n\n')

                    new_pq_paras = list(
                        map(lambda s: (s[re.search('\d+(\.)?', s).span()[1]:].strip(), sit_date, parl_no, section), # remove index, add sitting date, parl no, section
                            map(lambda s: re.sub('Column:(\s+)\d+', '', s.replace('  ', ' ').replace('\n', ' ').strip()), # remove column indicators and extra spaces
                                filter(lambda para: re.match(r'\d+(\.)?(\s+)(.|\s){8,75}(\s+)asked(\s+)the(\s+)', para), raw_paras)))) # keep only paragraphs that are pqs.
                    
                    pq_paras += new_pq_paras
                    print(str(len(new_pq_paras)) + ' ', end='')

                docs_saved += 1
                if docs_saved % 20 == 0:
                    print(f'Saved {docs_saved} / {num_results} docs at {datetime.now().strftime("%H%M")}')

                # Close tab         
                driver.close()
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(1)
            except Exception as e:
                print('exception' + str(e))
                import pdb
                pdb.set_trace()
                
                docs_saved += 1
                driver.close()
                driver.switch_to.window(driver.window_handles[1])
                time.sleep(1)
                continue
    else:
        docs_saved += 20
        docs_to_skip -= 20
        print('skipping')
        
    if docs_saved == num_results:
        break
    elif docs_saved == 20: # if we're on the first results page then there is no previous page button
        next_page = driver.find_element(
            by=By.XPATH, 
            value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/div[3]/section/ul/li[1]/a')
    else: # if we're not on the first results page then there is a previous page button
        next_page = driver.find_element(
            by=By.XPATH, 
            value='/html/body/app-root/app-result/div/div/div[2]/div/div/div/div/div/div[1]/div[3]/section/ul/li[3]/a')
    
    find_and_click(driver, next_page)
    pages_seen += 1
    print(f'Seen {pages_seen} / {num_pages} pages')

    # Sleep momentarily because next page takes a while to load     
    time.sleep(2) 


skipping
Seen 1 / 596 pages
skipping
Seen 2 / 596 pages
skipping
Seen 3 / 596 pages
skipping
Seen 4 / 596 pages
skipping
Seen 5 / 596 pages
skipping
Seen 6 / 596 pages
skipping
Seen 7 / 596 pages
skipping
Seen 8 / 596 pages
skipping
Seen 9 / 596 pages
skipping
Seen 10 / 596 pages
skipping
Seen 11 / 596 pages
skipping
Seen 12 / 596 pages
skipping
Seen 13 / 596 pages
skipping
Seen 14 / 596 pages
skipping
Seen 15 / 596 pages
skipping
Seen 16 / 596 pages
skipping
Seen 17 / 596 pages
skipping
Seen 18 / 596 pages
skipping
Seen 19 / 596 pages
skipping
Seen 20 / 596 pages
skipping
Seen 21 / 596 pages
skipping
Seen 22 / 596 pages
skipping
Seen 23 / 596 pages
skipping
Seen 24 / 596 pages
skipping
Seen 25 / 596 pages
skipping
Seen 26 / 596 pages
skipping
Seen 27 / 596 pages
skipping
Seen 28 / 596 pages
skipping
Seen 29 / 596 pages
skipping
Seen 30 / 596 pages
skipping
Seen 31 / 596 pages
skipping
Seen 32 / 596 pages
skipping
Seen 33 / 596 pages
skipping
Seen 34 / 596 pages
skipping
Seen 35 / 596 

ipdb>  continue


1 1 1 1 1 2 2 1 1 1 1 3 1 Saved 6120 / 11904 docs at 1254
Seen 306 / 596 pages
1 1 1 1 1 1 1 1 1 1 

In [ ]:
pq_paras_df = pd.DataFrame(pq_paras, columns=['para', 'sit_date', 'parl_no', 'section'])
pq_paras_df.to_csv('pq_paras.csv', index=False)

In [ ]:
docs_saved, len(pq_paras)